In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [11]:

data = pd.read_excel('Training data.xlsx')

x_train = np.array(data.iloc[:,0:8])
y_train = np.array(data.iloc[:,8]).reshape(-1,1)

def feature_changing(x):
    # Encoding categories to numbers
    x[:, 0] = np.where(x[:, 0] == 'yes', 1, 0)
    x[:, 1] = np.where(x[:, 1] == 'F', 1, 0)
    return x.astype(np.float64)

x_train = feature_changing(x_train)

In [12]:
def z_score(x_train):
    x_mean = np.mean(x_train, axis=0)
    x_std = np.std(x_train, axis=0)
    x_std[x_std == 0] = 1
    x_train = (x_train - x_mean) / x_std
    return x_train, x_std, x_mean



In [13]:
def cost(x_train, y_train, w, b):
    m = x_train.shape[0]
    y_pred = np.dot(x_train, w) + b
    loss = np.sum((y_pred - y_train)**2) / (2 * m)
    return loss

In [14]:
def gradient_descent(x_train, y_train, w, b):
    m = x_train.shape[0]
    learning_rate = 0.1

    y_pred = np.dot(x_train, w) + b
    error = y_pred - y_train

    dw = np.dot(x_train.T, error) / m
    db = np.sum(error) / m

    w = w - learning_rate * dw
    b = b - learning_rate * db
    return w, b

In [15]:
x_train = x_train.astype(np.float64)
x_train, x_std, x_mean = z_score(x_train)

np.random.seed(2147483647)
w = np.random.randn(x_train.shape[1], 1)
b = np.random.randn(1)

old_cost = 0

# Training loop
while abs(old_cost - cost(x_train, y_train, w, b)) > 0.00001:
    old_cost = cost(x_train, y_train, w, b)
    w, b = gradient_descent(x_train, y_train, w, b)


test_df = pd.read_excel('Test data.xlsx')
x_predict = test_df.iloc[:, :8].to_numpy()
x_predict = feature_changing(x_predict)
x_predict = (x_predict - x_mean) / x_std # Scale using train stats
ans = test_df.iloc[:, 8].to_numpy()

# Prediction and cleaning shape
y_predict = np.dot(x_predict, w) + b
y_predict = y_predict.flatten()

accuracy = 0
for dim in range(len(ans)):
    if abs(y_predict[dim] - ans[dim]) < 0.5:
        accuracy += 1

accuracy = round(accuracy * 100 / len(ans), 2)
ok = 'Congratulations' if accuracy > 95 else 'Optimization required'
print(f"{ok}, your accuracy is {accuracy}%")

Congratulations, your accuracy is 100.0%
